In [170]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
import pickle
import json
import os
import random
import tensorflow as tf

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import optimizers
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [171]:
print(keras.__version__)

2.0.6


In [172]:
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 4} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [173]:
data = pd.DataFrame([['fuck you, life is shitty', 0]], columns=['text', 'sent'])

In [174]:
df = pd.DataFrame([[1, 2], [3, 4]], columns=list('AB'))
df = df[df['B'] == 2]
df

,A,B
0,1,2


In [175]:
data_kaggle = pd.read_csv('training_data/KaggleTranslated.csv', encoding='latin_1', index_col = 0)
data_kaggle = data_kaggle.rename(index=str, columns={"SentimentText": "text"})

print(data_kaggle['sent'].value_counts())

diff = abs(data_kaggle['sent'].value_counts()[0] - data_kaggle['sent'].value_counts()[1])
print (diff)

data_kaggle_pos = data_kaggle[data_kaggle['sent'] == 1]

drop_indices = np.random.choice(data_kaggle_pos.index, diff, replace=False)

data_kaggle = data_kaggle.drop(drop_indices)

data_kaggle = data_kaggle.sample(frac=1).reset_index(drop=True)

data_kaggle_test = data_kaggle[data_kaggle.index < 30000]
data_kaggle_train = data_kaggle[data_kaggle.index >= 30000]
    
print(data_kaggle_train['sent'].value_counts())
print(data_kaggle_test['sent'].value_counts())

1    39822
0    31435
Name: sent, dtype: int64
8387
1    16493
0    16377
Name: sent, dtype: int64
0    15058
1    14942
Name: sent, dtype: int64


In [176]:
data_pol = pd.read_csv('training_data/training_set_sentipolc16.csv', encoding = "latin_1")
del data_pol['idtwitter']
del data_pol['iro']
del data_pol['top']
del data_pol['subj']

data_pol = data_pol.drop(data_pol[(data_pol['opos'] == 1) & (data_pol['oneg'] == 1)].index)
data_pol = data_pol.drop(data_pol[(data_pol['lpos'] == 1) & (data_pol['lneg'] == 1)].index)

data_pol.loc[data_pol['lpos'] == 1, 'sent'] = 1
data_pol.loc[data_pol['lneg'] == 1, 'sent'] = 0

data_pol.loc[data_pol['opos'] == 1, 'sent'] = 1
data_pol.loc[data_pol['oneg'] == 1, 'sent'] = 0


data_pol = data_pol[['text','sent']]

print(data_pol['sent'].value_counts())

diff = data_pol['sent'].value_counts()[0] - data_pol['sent'].value_counts()[1]
print (diff)
d_neg = data_pol[data_pol['sent'] == 0]

drop_indices = np.random.choice(d_neg.index, diff-100, replace=False)

data_pol = data_pol.drop(drop_indices)
    
print(data_pol['sent'].value_counts())

0.0    2472
1.0    1601
Name: sent, dtype: int64
871
0.0    1701
1.0    1601
Name: sent, dtype: int64


In [177]:
data_pol = data_pol.sample(frac=1).reset_index(drop=True)

data_null = data_pol[np.isnan(data_pol['sent'])]
drop_indices = np.random.choice(data_null.index, data_null.shape[0], replace=False)

data_pol = data_pol.drop(drop_indices)

data_pol = data_pol.sample(frac=1).reset_index(drop=True)

data_pol_test = data_pol[data_pol.index < 600]
data_pol_train = data_pol[data_pol.index >= 600]


print(data_pol_test['sent'].value_counts())
print(data_pol_train['sent'].value_counts())

1.0    314
0.0    286
Name: sent, dtype: int64
0.0    1415
1.0    1287
Name: sent, dtype: int64


In [178]:
data_pol = data_pol.sample(frac=1).reset_index(drop=True)
data = data.append(data_kaggle_train,ignore_index=True)
data = data.append(data_pol_train,  ignore_index=True)
data = data.append(data_pol_train,  ignore_index=True)

print(data['sent'].value_counts())

0.0    19208
1.0    19067
Name: sent, dtype: int64


In [179]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
data['text'] = data['text'].apply((lambda x: re.sub(':\)','',x)))
data['text'] = data['text'].apply((lambda x: re.sub(':\(','',x)))
data['text'] = data['text'].apply((lambda x: re.sub('#','',x)))
data['text'] = data['text'].apply((lambda x: re.sub('Berlusconi','akfha',x)))
data['text'] = data['text'].apply((lambda x: re.sub('Renzi','lafh',x)))
data['text'] = data['text'].apply((lambda x: re.sub('Salvini','kjahfka',x)))
data['text'] = data['text'].apply((lambda x: re.sub('Grillo','lhalahfl',x)))
data['text'] = data['text'].apply((lambda x: re.sub('berlusconi','akfha',x)))
data['text'] = data['text'].apply((lambda x: re.sub('renzi','lafh',x)))
data['text'] = data['text'].apply((lambda x: re.sub('salvini','kjahfka',x)))
data['text'] = data['text'].apply((lambda x: re.sub('grillo','lhalahfl',x)))
data['text'] = data['text'].apply((lambda x: re.sub('Lega','sds',x)))
data['text'] = data['text'].apply((lambda x: re.sub('PD ','fsfsfsf',x)))
data['text'] = data['text'].apply((lambda x: re.sub('FI ','dsgsgsgs',x)))

print(data['sent'].value_counts())

data = data.sample(frac=1).reset_index(drop=True)

num_words = 20000
maxlen = 30
tokenizer = Tokenizer(num_words=num_words, split=' ')
tokenizer.fit_on_texts(data['text'].values)
tweets = tokenizer.texts_to_sequences(data['text'].values)
tweets = pad_sequences(tweets, maxlen=maxlen)

tokenizer_file = open("model_files/tokenizer.pickle", "wb")
pickle.dump(tokenizer, tokenizer_file)
tokenizer_file.close()

0.0    19208
1.0    19067
Name: sent, dtype: int64


In [180]:
emb_dim = 128 #128
lstm_size = 196 #196
lstm2_size = 98

model = Sequential()
model.add(Embedding(num_words, emb_dim,input_length = tweets.shape[1]))
model.add(Dropout(0.5, noise_shape=None, seed=None))
model.add(LSTM(lstm_size, dropout=0.5, recurrent_dropout=0.5,return_sequences=True))
model.add(LSTM(lstm2_size, dropout=0.5, recurrent_dropout=0.5))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer= 'adam' ,metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 30, 128)           2560000   
_________________________________________________________________
dropout_10 (Dropout)         (None, 30, 128)           0         
_________________________________________________________________
lstm_19 (LSTM)               (None, 30, 196)           254800    
_________________________________________________________________
lstm_20 (LSTM)               (None, 98)                115640    
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 198       
Total params: 2,930,638
Trainable params: 2,930,638
Non-trainable params: 0
_________________________________________________________________
None


In [181]:
labels = pd.get_dummies(data['sent']).values
tweets_train, tweets_test, labels_train, labels_test = train_test_split(tweets,labels, test_size = 0.1, random_state = 42)
print(tweets_train.shape,labels_train.shape)
print(tweets_test.shape,labels_test.shape)

(34447, 30) (34447, 2)
(3828, 30) (3828, 2)


In [182]:
batch_size = 512
model.fit(tweets_train, labels_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
12s - loss: 0.6484 - acc: 0.6157
Epoch 2/7
10s - loss: 0.5340 - acc: 0.7374
Epoch 3/7
10s - loss: 0.4656 - acc: 0.7857
Epoch 4/7
10s - loss: 0.4198 - acc: 0.8107
Epoch 5/7
10s - loss: 0.3872 - acc: 0.8280
Epoch 6/7
10s - loss: 0.3650 - acc: 0.8423
Epoch 7/7
10s - loss: 0.3456 - acc: 0.8520


In [183]:
validation_size = 500

tweets_validate = tweets_test[0:validation_size]
labels_validate = labels_test[0:validation_size]
tweets_test = tweets_test[validation_size:]
labels_test = labels_test[validation_size:]
score,acc = model.evaluate(tweets_test, labels_test, verbose = 1, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

3072/3328 [==========================>...] - ETA: 0sscore: 0.57
acc: 0.74


In [184]:
score,acc = model.evaluate(tweets_validate, labels_validate, verbose = 1, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

500/500 [==============================] - 0s
score: 0.50
acc: 0.78


In [185]:
model.save('model_files/ItalianSentCls.h5')

In [186]:
tweets_kaggle = data_kaggle_test['text'].values
tweets_kaggle = tokenizer.texts_to_sequences(data_kaggle_test['text'].values)
tweets_kaggle = pad_sequences(tweets_kaggle, maxlen=maxlen)
labels_kaggle = pd.get_dummies(data_kaggle_test['sent']).values
score,acc = model.evaluate(tweets_kaggle, labels_kaggle, verbose = 1, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

29696/30000 [============================>.] - ETA: 0sscore: 0.61
acc: 0.72


In [192]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(tweets_kaggle)):    
    result = model.predict(tweets_kaggle[x].reshape(1,tweets_test.shape[1]),batch_size=1,verbose = 2)[0]   
    if random.uniform(0, 1) < 0.0003:
        print(result)
        print(x)   
    if result[0]>result[1]:
        conf = result[0]
    else:
        conf = result[1]   
    if conf > 0.80:
        if np.argmax(result) == np.argmax(labels_kaggle[x]):
            if np.argmax(labels_kaggle[x]) == 0:
                neg_correct += 1
            else:
                pos_correct += 1

        if np.argmax(labels_kaggle[x]) == 0:
            neg_cnt += 1
        else:
            pos_cnt += 1

print("pos_acc", pos_correct/pos_cnt*100, "%", pos_cnt)
print("neg_acc", neg_correct/neg_cnt*100, "%", neg_cnt)

[0.2184135  0.78158647]
2482
[0.04721038 0.95278966]
4158
[0.02263385 0.9773662 ]
6798
[0.03695268 0.96304727]
11987
[0.43978882 0.5602112 ]
12820
[0.04812514 0.9518749 ]
16333
[0.01602385 0.9839761 ]
17151
[0.64830303 0.35169703]
22198
[0.963292   0.03670794]
24619
[0.23398651 0.7660135 ]
29754
pos_acc 76.08521704340868 % 9998
neg_acc 81.07770627221208 % 10411


In [187]:
tweets_data_pol_test = data_pol_test['text'].values
tweets_data_pol_test = tokenizer.texts_to_sequences(data_pol_test['text'].values)
tweets_data_pol_test = pad_sequences(tweets_data_pol_test, maxlen=maxlen)
labels_data_pol_test = pd.get_dummies(data_pol_test['sent']).values
score,acc = model.evaluate(tweets_data_pol_test, labels_data_pol_test, verbose = 1, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

512/600 [========================>.....] - ETA: 0sscore: 0.61
acc: 0.74


In [188]:
tweets_data_pol_train = data_pol_train['text'].values
tweets_data_pol_train = tokenizer.texts_to_sequences(data_pol_train['text'].values)
tweets_data_pol_train = pad_sequences(tweets_data_pol_train, maxlen=maxlen)
labels_data_pol_train = pd.get_dummies(data_pol_train['sent']).values
diff = data_pol['sent'].value_counts()[0] - data_pol['sent'].value_counts()[1]
print (diff)
    
print(data_pol_train['sent'].value_counts())

100
0.0    1415
1.0    1287
Name: sent, dtype: int64


In [189]:
batch_size = 512
model.fit(tweets_data_pol_train, labels_data_pol_train, epochs = 2, batch_size=batch_size, verbose = 2)

Epoch 1/2
0s - loss: 0.2347 - acc: 0.9164
Epoch 2/2
0s - loss: 0.2107 - acc: 0.9152


In [190]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(tweets_data_pol_test)):
    
    result = model.predict(tweets_data_pol_test[x].reshape(1,tweets_test.shape[1]),batch_size=1,verbose = 2)[0]
    
    if random.uniform(0, 1) < 0.001:
        print(result)
        print(x)
    
    if result[0]>result[1]:
        conf = result[0]
    else:
        conf = result[1]
   
    if conf > 0.80:
        if np.argmax(result) == np.argmax(labels_data_pol_test[x]):
            if np.argmax(labels_data_pol_test[x]) == 0:
                neg_correct += 1
            else:
                pos_correct += 1

        if np.argmax(labels_kaggle[x]) == 0:
            neg_cnt += 1
        else:
            pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%", pos_cnt)
print("neg_acc", neg_correct/neg_cnt*100, "%", neg_cnt)

[0.5665237  0.43347627]
22
[0.97419894 0.025801  ]
54
pos_acc 85.86387434554975 % 191
neg_acc 78.80184331797236 % 217


In [191]:
text = 'Questa recensione sarebbe incredibilmente brutta e cattiva'
text = [text]
text = tokenizer.texts_to_sequences(text)
text = pad_sequences(text, maxlen = 30)

print(model.predict(text))

[[0.8881522  0.11184778]]
